# Lab-1 : Attension

Before, I address what is self-attension ? first we review how RNN maintaining a hidden state allows to incorporate its representation of previous words/vectors it has processed with the current one it’s processing. Thus, Self attension is the method used by transformer for understanding the relevence of other words from vocabulary to the currect processing word in a given sentence.

### Task-0 

In [1]:
# python from scratch library
import math
import numpy as np 

# 
import torch
import torch.nn as nn 
import torch.nn.functional as F 


### Task-1

Attension - In a given sentence, It keep track every single word have some kind of affinity towards other word or not

$$ \text{Attention}(Q, K, V) = \text{softmax}\left(\frac{QK^T}{\sqrt{d_k}}\right)V $$



In [2]:
# compute the query, key and value matrices
def softmax(x):
    return ( np.exp(x).T / np.sum(np.exp(x), axis=-1)).T

def scaled_dot_product_attention( q, k, v, mask=None): # mask is none by default and is in encoder setting which 
                                                       # can be changed to decoder by passing mask
    d_k=q.shape[-1]
    scaled= np.matmul(q, k.T)/math.sqrt(d_k)
    if mask is not None:
        scaled= scaled+mask
    print("scaled", scaled)
    attention= softmax(scaled)
    out=np.matmul(attention, v)
    return out, attention # new values , attention

In [3]:
# length of input sequence
L=4
# size of thes vectors
d_k = 8
d_v = 8
# initialize the q k v matrices
q = np.random.rand(L, d_k)
k = np.random.rand(L, d_k)
v = np.random.rand(L, d_v)

# compute the attention
out, attention = scaled_dot_product_attention(q, k, v)
# print the output and attention
print("Output:\n", out)
print("Attention:\n", attention)

# create a mask for the decoder - prediction of the future tokens
mask= np.tril(np.ones((L,L)))
mask[mask==0]=-np.inf
mask[mask==1]=0

# compute the attention with mask
out_masked, attention_masked = scaled_dot_product_attention(q, k, v, mask)
# print the output and attention with mask
print("Output with mask:\n", out_masked)
print("Attention with mask:\n", attention_masked)


scaled [[0.87131859 0.86252614 0.48247449 0.6241726 ]
 [1.02132395 1.12710965 0.9030035  1.07739794]
 [1.02807672 0.91894661 0.71476049 0.99712315]
 [0.79079892 0.78990133 0.61787771 0.64059389]]
Output:
 [[0.39125901 0.6638908  0.44900505 0.42719732 0.66951825 0.6359097
  0.63514343 0.36880897]
 [0.41216041 0.69503973 0.4280375  0.400267   0.68886302 0.61062178
  0.61598543 0.37990854]
 [0.40441745 0.67985271 0.43528608 0.42071218 0.68552592 0.6053806
  0.62497903 0.38505464]
 [0.38840181 0.68091523 0.44167785 0.42328641 0.67909902 0.62552722
  0.63767863 0.36375848]]
Attention:
 [[0.2898456  0.28730832 0.19646893 0.22637715]
 [0.2464468  0.27394623 0.21894612 0.26066085]
 [0.27799762 0.24925649 0.20322146 0.26952442]
 [0.27020906 0.26996663 0.22730091 0.23252341]]
scaled [[0.87131859       -inf       -inf       -inf]
 [1.02132395 1.12710965       -inf       -inf]
 [1.02807672 0.91894661 0.71476049       -inf]
 [0.79079892 0.78990133 0.61787771 0.64059389]]
Output with mask:
 [[0.0417

### Task-2 : Multi-Head Attension

In [7]:
def scaled_dot_product(q, k, v, mask=None):
    d_k = q.size()[-1]
    scaled = torch.matmul(q, k.transpose(-1, -2)) / math.sqrt(d_k)
    if mask is not None:
        scaled += mask
    attention = F.softmax(scaled, dim=-1)
    values = torch.matmul(attention, v)
    return values, attention

In [8]:
sequence_length = 4 # My name is Ankit
batch_size = 1 # number of sentences in batch
input_dim = 512 # vector dimension of each input
d_model = 512 # vector dimension of each output  of each attention unit       
x = torch.rand((batch_size, sequence_length, input_dim)) # input tensor

qkv_layer = nn.Linear(input_dim, 3*d_model) # here we are concatenating q, k, v in one matrix which will be split later
qkv = qkv_layer(x)

num_heads= 8 # now we have 8 attention head
head_dim= d_model//num_heads # each head will have dim of 512 / 8 => 64
qkv = qkv.reshape(batch_size, sequence_length, num_heads, 3*head_dim)

# let's change sequencing in qkv just to make things easy in future and perform parallel operations of last 2 dimensions
qkv=qkv.permute(0, 2, 1, 3) # [batch_size, num_heads, sequence_length, 3*head_dim]
qkv.shape

# now we will divide the chuck of each head into q, k, v
q, k, v=qkv.chunk(3, dim=-1) # divide last dimension by 3 i.e 192/3 = 64

values, attention = scaled_dot_product(q, k, v)


In [9]:

class MultiHeadAttention(nn.Module):

    def __init__(self, input_dim, d_model, num_heads):
        super().__init__()
        self.input_dim = input_dim
        self.d_model = d_model
        self.num_heads = num_heads
        self.head_dim = d_model // num_heads
        self.qkv_layer = nn.Linear(input_dim , 3 * d_model)
        self.linear_layer = nn.Linear(d_model, d_model)
    
    def forward(self, x, mask=None): # forward pass
        
        batch_size, sequence_length, input_dim = x.size()
        print(f"x.size(): {x.size()}\n")
        print("-"*50)

        qkv = self.qkv_layer(x)
        print(f"qkv.size(): {qkv.size()}\n")
        print("-"*50)

        qkv = qkv.reshape(batch_size, sequence_length, self.num_heads, 3 * self.head_dim)
        print(f"qkv.size(): {qkv.size()}")
        qkv = qkv.permute(0, 2, 1, 3)
        print(f"qkv.size(): {qkv.size()}\n")
        print("-"*50)

        q, k, v = qkv.chunk(3, dim=-1)
        print(f"q size: {q.size()}     k size: {k.size()}     v size: {v.size()}\n")
        print("-"*50)

        values, attention = scaled_dot_product(q, k, v, mask)
        print(f"values.size(): {values.size()}          attention.size:{ attention.size()} ")
        print("-"*50)
        values = values.reshape(batch_size, sequence_length, self.num_heads * self.head_dim)
        print(f"values.size(): {values.size()}\n")
        print("-"*50)

        out = self.linear_layer(values)
        print(f"out.size(): {out.size()}\n")
        print("-"*50)
        return out

In [10]:
input_dim = 512
d_model = 512
num_heads = 8

batch_size = 30
sequence_length = 5
x = torch.randn( (batch_size, sequence_length, input_dim) )

model = MultiHeadAttention(input_dim, d_model, num_heads)
out = model.forward(x)

x.size(): torch.Size([30, 5, 512])

--------------------------------------------------
qkv.size(): torch.Size([30, 5, 1536])

--------------------------------------------------
qkv.size(): torch.Size([30, 5, 8, 192])
qkv.size(): torch.Size([30, 8, 5, 192])

--------------------------------------------------
q size: torch.Size([30, 8, 5, 64])     k size: torch.Size([30, 8, 5, 64])     v size: torch.Size([30, 8, 5, 64])

--------------------------------------------------
values.size(): torch.Size([30, 8, 5, 64])          attention.size:torch.Size([30, 8, 5, 5]) 
--------------------------------------------------
values.size(): torch.Size([30, 5, 512])

--------------------------------------------------
out.size(): torch.Size([30, 5, 512])

--------------------------------------------------


## Reference 
1. https://colab.research.google.com/github/tensorflow/tensor2tensor/blob/master/tensor2tensor/notebooks/hello_t2t.ipynb
2. 